In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier

import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
import sklearn
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,Concatenate,concatenate, Input
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import pickle

In [2]:
list_string=['ucddb002','ucddb003','ucddb005','ucddb006','ucddb007','ucddb009',\
             'ucddb010','ucddb012','ucddb014','ucddb015','ucddb017',\
             'ucddb019','ucddb020','ucddb021','ucddb022','ucddb023','ucddb024',\
             'ucddb025','ucddb026','ucddb027','ucddb028']#'ucddb008','ucddb011','ucddb013','ucddb014',


valid_features=np.zeros((1,1408))
test_features=np.zeros((1,1408))
valid_labels=np.zeros((1,1))
test_labels=np.zeros((1,1))
for l in list_string:
        
        
        ecg_valid = loadmat('D:\\CSE499\\files2\\'+l+'_ecg_valid.mat')
        ecg_valid = np.array(ecg_valid['ecg_valid'])
        ecg_valid_labels=loadmat('D:\\CSE499\\files2\\'+l+'_valid_labels.mat')
        ecg_valid_labels = np.array(ecg_valid_labels['class_valid'])
        valid_features=np.append(valid_features,ecg_valid,axis=0)
        valid_labels=np.append(valid_labels,ecg_valid_labels)
       
        
        ecg_test = loadmat('D:\\CSE499\\files2\\'+l+'_ecg_test.mat')
        ecg_test = np.array(ecg_test['ecg_test'])
        ecg_test_labels=loadmat('D:\\CSE499\\files2\\'+l+'_test_labels.mat')
        ecg_test_labels = np.array(ecg_test_labels['class_test'])
        test_features=np.append(test_features,ecg_test,axis=0)
        test_labels=np.append(test_labels,ecg_test_labels)
        
ecg_valid=valid_features[1:,:]
valid_labels=valid_labels[1:]
valid_labels = valid_labels.flatten()

ecg_test=test_features[1:,:]
test_labels=test_labels[1:]
test_labels = test_labels.flatten()

In [3]:
train_features=np.zeros((1,1408))

train_labels=np.zeros((1,1))

for l in list_string:
        ecg_train = loadmat('D:\\CSE499\\files2\\'+l+'_ecg_train.mat')
        ecg_train = np.array(ecg_train['ecg_train'])
        ecg_train_labels=loadmat('D:\\CSE499\\files2\\'+l+'_train_labels.mat')
        ecg_train_labels = np.array(ecg_train_labels['class_train'])
        train_features=np.append(train_features,ecg_train,axis=0)
        train_labels=np.append(train_labels,ecg_train_labels)
        
ecg_train=train_features[1:,:]

train_labels=train_labels[1:]
train_labels = train_labels.flatten()

In [4]:
ecg_train_mean=np.mean(ecg_train)
ecg_train_std=np.std(ecg_train)

In [5]:
for i in range(ecg_train.shape[0]):
    ecg_train[i,:]=(ecg_train[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_valid.shape[0]):
    ecg_valid[i,:]=(ecg_valid[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_test.shape[0]):
    ecg_test[i,:]=(ecg_test[i,:]-ecg_train_mean)/ecg_train_std
    

In [6]:
y_train=train_labels
y_valid=valid_labels
y_test=test_labels

In [7]:
num_classes=2

In [8]:
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = tensorflow.keras.utils.to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = tensorflow.keras.utils.to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = tensorflow.keras.utils.to_categorical(y_test_num, num_classes)

In [9]:
print("Num GPUs Available: ", len(tensorflow.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [10]:
from sklearn.linear_model import LogisticRegression

# Create and train the Logistic Regression model
logistic_regression_model = LogisticRegression(random_state=42)
logistic_regression_model.fit(ecg_train, y_train)

# Make predictions on the test set
logistic_regression_predictions = logistic_regression_model.predict(ecg_test)

# Evaluate the model
logistic_regression_accuracy = accuracy_score(y_test, logistic_regression_predictions)
logistic_regression_conf_matrix = confusion_matrix(y_test, logistic_regression_predictions)
logistic_regression_specificity = logistic_regression_conf_matrix[0, 0] / (logistic_regression_conf_matrix[0, 0] + logistic_regression_conf_matrix[0, 1])
logistic_regression_sensitivity = logistic_regression_conf_matrix[1, 1] / (logistic_regression_conf_matrix[1, 1] + logistic_regression_conf_matrix[1, 0])

# Print or use these metrics for Logistic Regression as needed
print("Logistic Regression Accuracy:", logistic_regression_accuracy)
print("Logistic Regression Confusion Matrix:\n", logistic_regression_conf_matrix)
print("Logistic Regression Specificity:", logistic_regression_specificity)
print("Logistic Regression Sensitivity:", logistic_regression_sensitivity)


D:\CSE299\Flutter\Anaconda\envs\EEG_Signal\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.6571826194526056
Logistic Regression Confusion Matrix:
 [[33501 16867]
 [  869   499]]
Logistic Regression Specificity: 0.6651246823379924
Logistic Regression Sensitivity: 0.364766081871345


In [11]:
from sklearn.tree import DecisionTreeClassifier

# Create and train the Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(ecg_train, y_train)

# Make predictions on the test set
decision_tree_predictions = decision_tree_model.predict(ecg_test)

# Evaluate the model
decision_tree_accuracy = accuracy_score(y_test, decision_tree_predictions)
decision_tree_conf_matrix = confusion_matrix(y_test, decision_tree_predictions)
decision_tree_specificity = decision_tree_conf_matrix[0, 0] / (decision_tree_conf_matrix[0, 0] + decision_tree_conf_matrix[0, 1])
decision_tree_sensitivity = decision_tree_conf_matrix[1, 1] / (decision_tree_conf_matrix[1, 1] + decision_tree_conf_matrix[1, 0])

# Print or use these metrics for Decision Tree as needed
print("Decision Tree Accuracy:", decision_tree_accuracy)
print("Decision Tree Confusion Matrix:\n", decision_tree_conf_matrix)
print("Decision Tree Specificity:", decision_tree_specificity)
print("Decision Tree Sensitivity:", decision_tree_sensitivity)


Decision Tree Accuracy: 0.9440814906448122
Decision Tree Confusion Matrix:
 [[48747  1621]
 [ 1272    96]]
Decision Tree Specificity: 0.9678168678526048
Decision Tree Sensitivity: 0.07017543859649122


In [12]:
from sklearn.ensemble import AdaBoostClassifier

# Create and train the AdaBoost model
adaboost_model = AdaBoostClassifier(random_state=42)
adaboost_model.fit(ecg_train, y_train)

# Make predictions on the test set
adaboost_predictions = adaboost_model.predict(ecg_test)

# Evaluate the model
adaboost_accuracy = accuracy_score(y_test, adaboost_predictions)
adaboost_conf_matrix = confusion_matrix(y_test, adaboost_predictions)
adaboost_specificity = adaboost_conf_matrix[0, 0] / (adaboost_conf_matrix[0, 0] + adaboost_conf_matrix[0, 1])
adaboost_sensitivity = adaboost_conf_matrix[1, 1] / (adaboost_conf_matrix[1, 1] + adaboost_conf_matrix[1, 0])

# Print or use these metrics for AdaBoost as needed
print("AdaBoost Accuracy:", adaboost_accuracy)
print("AdaBoost Confusion Matrix:\n", adaboost_conf_matrix)
print("AdaBoost Specificity:", adaboost_specificity)
print("AdaBoost Sensitivity:", adaboost_sensitivity)


AdaBoost Accuracy: 0.6210955620844286
AdaBoost Confusion Matrix:
 [[31533 18835]
 [  768   600]]
AdaBoost Specificity: 0.6260522554002541
AdaBoost Sensitivity: 0.43859649122807015


In [13]:
import xgboost as xgb

# Create and train the XGBoost model
xgboost_model = xgb.XGBClassifier(random_state=42)
xgboost_model.fit(ecg_train, y_train)

# Make predictions on the test set
xgboost_predictions = xgboost_model.predict(ecg_test)

# Evaluate the model
xgboost_accuracy = accuracy_score(y_test, xgboost_predictions)
xgboost_conf_matrix = confusion_matrix(y_test, xgboost_predictions)
xgboost_specificity = xgboost_conf_matrix[0, 0] / (xgboost_conf_matrix[0, 0] + xgboost_conf_matrix[0, 1])
xgboost_sensitivity = xgboost_conf_matrix[1, 1] / (xgboost_conf_matrix[1, 1] + xgboost_conf_matrix[1, 0])

# Print or use these metrics for XGBoost as needed
print("XGBoost Accuracy:", xgboost_accuracy)
print("XGBoost Confusion Matrix:\n", xgboost_conf_matrix)
print("XGBoost Specificity:", xgboost_specificity)
print("XGBoost Sensitivity:", xgboost_sensitivity)


XGBoost Accuracy: 0.9243853409618061
XGBoost Confusion Matrix:
 [[47624  2744]
 [ 1168   200]]
XGBoost Specificity: 0.9455209656925032
XGBoost Sensitivity: 0.14619883040935672


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create and train the Gradient Boosting model
gradient_boosting_model = GradientBoostingClassifier(random_state=42)
gradient_boosting_model.fit(ecg_train, y_train)

# Make predictions on the test set
gradient_boosting_predictions = gradient_boosting_model.predict(ecg_test)

# Evaluate the model
gradient_boosting_accuracy = accuracy_score(y_test, gradient_boosting_predictions)
gradient_boosting_conf_matrix = confusion_matrix(y_test, gradient_boosting_predictions)
gradient_boosting_specificity = gradient_boosting_conf_matrix[0, 0] / (gradient_boosting_conf_matrix[0, 0] + gradient_boosting_conf_matrix[0, 1])
gradient_boosting_sensitivity = gradient_boosting_conf_matrix[1, 1] / (gradient_boosting_conf_matrix[1, 1] + gradient_boosting_conf_matrix[1, 0])

# Print or use these metrics for Gradient Boosting as needed
print("Gradient Boosting Accuracy:", gradient_boosting_accuracy)
print("Gradient Boosting Confusion Matrix:\n", gradient_boosting_conf_matrix)
print("Gradient Boosting Specificity:", gradient_boosting_specificity)
print("Gradient Boosting Sensitivity:", gradient_boosting_sensitivity)
